# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

# Индексация
## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [1]:
#from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors
import re
import os
from tqdm import tqdm_notebook as tqdm
import pymorphy2

from gensim.models.doc2vec import Doc2Vec, TaggedDocument, TaggedLineDocument
from nltk.tokenize import word_tokenize
import nltk
import pickle

import numpy as np
import json

import operator

C:\Users\1\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [11]:
def preprocessing(pars):
    for i in range(0, len(pars)):
        pars[i] = pars[i].replace('.', ' ')
        pars[i] = pars[i].replace(',', ' ')
        pars[i] = pars[i].replace(':', ' ')
        pars[i] = pars[i].replace(';', ' ')
        pars[i] = pars[i].replace('-', ' ')
        pars[i] = pars[i].replace(')', ' ')
        pars[i] = pars[i].replace('(', ' ')
        pars[i] = pars[i].replace('\\', ' ')
        pars[i] = pars[i].replace('/', ' ')
        pars[i] = pars[i].replace('"', ' ')
        pars[i] = pars[i].replace('!', ' ')
        pars[i] = pars[i].replace('?', ' ')
        pars[i] = pars[i].replace('\n', ' ')
        pars[i] = re.sub('[0-9]','', pars[i])
        pars[i] = pars[i].replace('   ', ' ')
        pars[i] = pars[i].replace('  ', ' ')
        pars[i] = pars[i].strip(' ')
    return pars

In [13]:
model_path = r"C:\Users\1\Downloads\all.norm-sz100-w10-cb0-it1-min100.w2v"
model = KeyedVectors.load_word2vec_format(model_path, binary=True, unicode_errors='ignore')

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [3]:
path = 'C:\\Users\\1\\Downloads\\Telegram Desktop\\articles\\article'

In [7]:
def make_pars(path):
    file = open(path, 'r', encoding = 'UTF-8')
    f = file.read()
    file.close()
    f = f+'. '
    f = f.lower()
    pars = re.findall('.*?[^А-Я|г]\. ', f)
    pars = preprocessing(pars) 
    return pars

In [5]:
def par_t_v(par):
    vecs = np.zeros(100)
    par = list(filter(None, par.split(' ')))
    for word in par:
        try:
            vec = model.get_vector(word)
        except Exception as ex:
            print(ex)
            continue
        if vecs.size is 0:
            vecs = vec
        else:
            vecs = vecs+vec
    vec = vecs/len(par)
    return vec

In [111]:
def write_json(pars_and_vecs, global_num):
    p = r'C:\Users\1\Downloads\Telegram Desktop\articles\json\all_json_w2v_' + str(global_num) +'.txt'
    with open(p, 'w') as f:
        json.dump(pars_and_vecs, f)
    return

In [110]:
def get_w2v_vectors(path):
    pars = make_pars(path)
    pars_and_vecs = []
    for i in range(0, len(pars)):
        vec_par = par_t_v(pars[i])
        pars_and_vecs.append(vec_par.tolist())
    return pars_and_vecs

In [ ]:
global_dict = {}
global_num = 0
path = r"C:\Users\1\Downloads\Telegram Desktop\articles\article"
for idx in tqdm(range(1, 2001)):
    path_file=path+"\\"+str(idx)+'.txt'
    if os.path.isfile(path_file):
        global_dict[str(idx)] = get_w2v_vectors(path_file)
        
    if idx%100 is 0:
        global_num+=1
        write_json(global_dict, global_num)
        global_dict = {} 

## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

У меня были какие-то проблемы с обучением модели. Она обучалась только на очень маленьких эпохах. На 150 простояла два дня, но так и недообучилась. Поэтому взяла вашу модель.

In [4]:
model_d = Doc2Vec.load('Doc2Vec_100s_1000e')

In [5]:
vector = model_d.infer_vector(["ходатайство"])
vector

array([-0.4282251 ,  0.23708375, -0.45930117,  0.10611931,  0.62889421,
        0.1196252 , -0.02407027,  0.45586878,  0.4159039 , -0.12778939,
       -0.00203439, -0.07862893,  0.62620658, -0.12105814,  0.10052878,
       -0.12932751,  0.18158963,  0.25525215, -0.532444  ,  0.0990846 ,
        0.10783657,  0.59053999,  0.02449054, -0.21601641,  0.39617512,
       -0.11783301,  0.01439113,  0.31621888,  0.27657464, -0.13583528,
       -0.0075618 , -0.02023337,  0.14193198,  0.01083831,  0.18012066,
        0.10876582, -0.20693055,  0.19861947,  0.09373802, -0.23966387,
        0.04996694,  0.54767364, -0.33492318, -0.40149394, -0.15552157,
       -0.18683743,  0.07880542, -0.51807427,  0.30847964, -0.05967191,
       -0.03973081,  0.34769812,  0.15159838,  0.18458614, -0.13132106,
        0.2101123 , -0.62978256, -0.27266434, -0.11287375,  0.38861862,
        0.17073986, -0.10981907, -0.17872557, -0.40869817,  0.36989751,
       -0.44914404, -0.02076871, -0.33414879, -0.22244197,  0.40

### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [6]:
def make_pars(path):
    file = open(path, 'r', encoding = 'UTF-8')
    f = file.read()
    file.close()
    f = f+'. '
    f = f.lower()
    pars = re.findall('.*?[^А-Я|г]\. ', f)
    pars = preprocessing(pars) 
    return pars

In [7]:
def doc_t_v(par):
    vec = np.zeros(100)
    try:
        vec = model_d.infer_vector(par)
    except Exception as ex:
        print(ex)
    return vec

In [8]:
def get_d2v_vectors(path):
    pars = make_pars(path)
    pars_and_vecs = []
    for i in range(0, len(pars)):
        vec_par = doc_t_v(pars[i])
        pars_and_vecs.append(vec_par.tolist())
    return pars_and_vecs

In [9]:
def write_json_d(pars_and_vecs, global_num):
    p = r'C:\Users\1\Downloads\Telegram Desktop\articles\json\all_json_d2v_' + str(global_num) +'.txt'
    with open(p, 'w') as f:
        json.dump(pars_and_vecs, f)
    return

In [12]:
global_dict = {}
global_num = 0
path = r"C:\Users\1\Downloads\Telegram Desktop\articles\article"
for idx in tqdm(range(1, 2001)):
    path_file=path+"\\"+str(idx)+'.txt'
    if os.path.isfile(path_file):
        global_dict[str(idx)] = get_d2v_vectors(path_file)
        
    if idx%100 is 0:
        global_num+=1
        write_json_d(global_dict, global_num)
        global_dict = {} 

A Jupyter Widget

# Функция поиска

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [58]:
def search_w2v(query):
    query = preprocessing(query.lower().split())
    query_vec = np.zeros(100)
    for word in query:
        vec = model.get_vector(word)
        query_vec += vec
    query_vec = query_vec / len(query)
    path = r"C:\Users\1\Downloads\Telegram Desktop\articles\json\all_json_w2v_"
    doc2similar = {}
    for idx in tqdm(range(1, 21)):
        with open(path + str(idx) + ".txt") as f:
            doc = json.load(f)
        for key, value in doc.items():
            doc2similar[key] = max(model.cosine_similarities(query_vec, [np.array(item) for item in value]))
    

    sorted_dict = sorted(doc2similar.items(), key=operator.itemgetter(1), reverse=True)
    num = sorted_dict[0][0]
    path = r"C:\Users\1\Downloads\Telegram Desktop\articles\article\\"
    with open(path + str(num) + ".txt", encoding = "UTF-8") as f:
        text = f.read()
    return text

In [59]:
query = "В  передаче дела по иску в части взыскания расходов, связанных с предоставлением льгот по оплате услуг отдельным категориям граждан, для пересмотра в порядке надзора судебных актов отказано, так как, удовлетворяя иск, суд исходил из того, что меры социальной поддержки, предусмотренные федеральными законами о социальной защите, являются расходными обязательствами Российской Федерации"
text = search_w2v(query)

A Jupyter Widget

C:\Users\1\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:920: RuntimeWarning: invalid value encountered in true_divide
  similarities = dot_products / (norm * all_norms)


In [60]:
text

'\n\nОпределение ВАС РФ от 22.12.2011 N ВАС-16916/11 по делу N А76-6807/2011  В  передаче дела по иску в части взыскания расходов, связанных с предоставлением льгот по оплате жилищно-коммунальных услуг отдельным категориям граждан, для пересмотра в порядке надзора судебных актов отказано, так как, удовлетворяя иск, суд исходил из того, что меры социальной поддержки, предусмотренные федеральными законами о социальной защите, являются расходными обязательствами Российской Федерации.'

In [14]:
def search_d2v(query):
    query = preprocessing(query.lower().split())
    query = ' '.join(query)
    query_vec = np.zeros(100)
    query_vec = model_d.infer_vector(query)

    path = r"C:\Users\1\Downloads\Telegram Desktop\articles\json\all_json_d2v_"
    doc2similar = {}
    for idx in tqdm(range(1, 21)):
        with open(path + str(idx) + ".txt") as f:
            doc = json.load(f)
        for key, value in doc.items():
            doc2similar[key] = max(model.cosine_similarities(query_vec, [np.array(item) for item in value]))
    

    sorted_dict = sorted(doc2similar.items(), key=operator.itemgetter(1), reverse=True)
    num = sorted_dict[0][0]
    path = r"C:\Users\1\Downloads\Telegram Desktop\articles\article\\"
    with open(path + str(num) + ".txt", encoding = "UTF-8") as f:
        text = f.read()
    return text

In [15]:
query = "В  передаче дела по иску в части взыскания расходов, связанных с предоставлением льгот по оплате услуг отдельным категориям граждан, для пересмотра в порядке надзора судебных актов отказано, так как, удовлетворяя иск, суд исходил из того, что меры социальной поддержки, предусмотренные федеральными законами о социальной защите, являются расходными обязательствами Российской Федерации"
text = search_d2v(query)

A Jupyter Widget

In [16]:
text

'\n\nКоллегия судей Высшего Арбитражного Суда Российской Федерации в составе председательствующего судьи Харчиковой Н.П., судей Бондаренко С.П.,  Марамышкиной  И.М. рассмотрела в судебном заседании заявление Государственной корпорации "Банк развития и внешнеэкономической деятельности" (Москва) от 21.11.2011 N 1380/050009 о пересмотре в порядке надзора решения Арбитражного суда города Москвы от 22.02.2011 по делу N А40-102283/10-120-420, постановления Девятого арбитражного апелляционного  суда от 12.05.2011 и постановления Федерального арбитражного суда Московского округа от 22.08.2011 по тому же делу по заявлению Самарской области в лице Министерства управления финансами Самарской области (г. Самара, далее - министерство управления финансами) о признании незаконным бездействия Государственной корпорации "Банк развития и внешнеэкономической деятельности" (Москва, далее - Внешэкономбанк), связанного с отказом осуществить полномочия, предусмотренные Порядком списания и восстановления в уч

In [17]:
def search(query, search_method):
    if search_method == 'word2vec':
        search_result = search_w2v(query)
    elif search_method == 'doc2vec':
        search_result = search_d2v(query)
    else:
        raise TypeError('unsupported search method')
    return search_result